# 导包

In [ ]:
#导包
from tensorflow.keras import layers, models
from tensorflow.keras.layers import ConvLSTM2D,LSTM, BatchNormalization, LayerNormalization,Input, Conv3D, TimeDistributed, Flatten
from tensorflow.keras.layers import Concatenate,Conv2D,TimeDistributed, MaxPooling2D, Input, MaxPooling3D
from tensorflow.keras.layers import  Reshape,Multiply
from tensorflow.keras.layers import Layer,Lambda,Dot,ReLU, Dense, Dropout, Activation, Flatten,Attention
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import r2_score,accuracy_score,precision_score
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras import regularizers
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import models, layers
# from scikeras.wrappers import KerasRegressor
import tensorflow.keras as keras
from tensorflow.keras import layers, Sequential, regularizers
# calculate RMSE
from tensorflow.keras.losses import MeanSquaredError
from math import sqrt
import pandas as pd 
import tensorflow as tf
import gc 
import gzip
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import os 
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # 指定使用 GPU 1
# from model import create_model
from evaluate import look_loss
from evaluate import all_estimate

gc.collect()

# 模型预测
# 在模型代码开头添加
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.optimizer.set_jit(True)  # 启用XLA编译

# 混合精度配置
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# 启用TF32加速（Ada特有）
tf.config.optimizer.set_experimental_options({
    'enable_tensorrt': True,
    'precision_mode': 'TF32'
})

In [ ]:
depth = [5.1, 18.5, 40.3, 65.8, 77.9, 92.3, 109.7, 130.7, 155.9, 186.1, 222.5, 266.0, 318.1, 380.2, 453.9, 541.1, 643.6, 763.3, 902.3, 1062.4]
data_st_processed = np.load(r'./data_st_processed.npy')
data_surface_processed = np.load(r'./data_surface_processed.npy')

# Layer2

In [ ]:
# data_st_processed = np.load(r'./data_st_processed.npy')
# data_surface_processed = np.load(r'./data_surface_processed.npy')
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
data_underwater = np.load(r'./layer1_pred_data.npy')
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,1:2] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer2_st_no.keras',      safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:6]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer2_st = scaler_l.inverse_transform(result)
y_test_layer2_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer2_unscaled[:,0],predict_layer2_st[:,0])
print('layer2 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer2_unscaled[:,0]-predict_layer2_st[:,0])))
print('layer2 Test mae: %.3f' % mae)

layer2_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer2_pred_data[non_nan_mask] = predict_layer2_st
true_data[non_nan_mask] = y_test_layer2_unscaled

layer2_pred_data[layer2_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer2/results.txt", "w") as file:
#     file.write(f"layer2 R² score: {r2}\n")
#     file.write(f"layer2 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer2_pred_data = layer2_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer2_pred_data.npy',layer2_pred_data)
# np.save(r'./layer2/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer2_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer2_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer3

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy')
layer2_fanyan = np.load(r'./layer2_pred_data.npy')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan), axis=-1)
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,2:3] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer3_st_no.keras',      safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:7]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer3_st = scaler_l.inverse_transform(result)
y_test_layer3_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer3_unscaled[:,0],predict_layer3_st[:,0])
print('layer3 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer3_unscaled[:,0]-predict_layer3_st[:,0])))
print('layer3 Test mae: %.3f' % mae)

layer3_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer3_pred_data[non_nan_mask] = predict_layer3_st
true_data[non_nan_mask] = y_test_layer3_unscaled

layer3_pred_data[layer3_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer3/results.txt", "w") as file:
#     file.write(f"layer3 R² score: {r2}\n")
#     file.write(f"layer3 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer3_pred_data = layer3_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer3_pred_data.npy',layer3_pred_data)
# np.save(r'./layer3/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer3_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer3_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer4

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy')
layer2_fanyan = np.load(r'./layer2_pred_data.npy')
layer3_fanyan = np.load(r'./layer3_pred_data.npy')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan,layer3_fanyan), axis=-1)
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,3:4] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer4_st_no.keras',      safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:8]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer4_st = scaler_l.inverse_transform(result)
y_test_layer4_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer4_unscaled[:,0],predict_layer4_st[:,0])
print('layer4 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer4_unscaled[:,0]-predict_layer4_st[:,0])))
print('layer4 Test mae: %.3f' % mae)

layer4_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer4_pred_data[non_nan_mask] = predict_layer4_st
true_data[non_nan_mask] = y_test_layer4_unscaled

layer4_pred_data[layer4_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer4/results.txt", "w") as file:
#     file.write(f"layer4 R² score: {r2}\n")
#     file.write(f"layer4 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer4_pred_data = layer4_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer4_pred_data.npy',layer4_pred_data)
# np.save(r'./layer4/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer4_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer4_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer5

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy')
layer2_fanyan = np.load(r'./layer2_pred_data.npy')
layer3_fanyan = np.load(r'./layer3_pred_data.npy')
layer4_fanyan = np.load(r'./layer4_pred_data.npy')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan,layer3_fanyan,layer4_fanyan), axis=-1)
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,4:5] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer5_st_no.keras',      safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:9]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer5_st = scaler_l.inverse_transform(result)
y_test_layer5_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer5_unscaled[:,0],predict_layer5_st[:,0])
print('layer5 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer5_unscaled[:,0]-predict_layer5_st[:,0])))
print('layer5 Test mae: %.3f' % mae)

layer5_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer5_pred_data[non_nan_mask] = predict_layer5_st
true_data[non_nan_mask] = y_test_layer5_unscaled

layer5_pred_data[layer5_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer5/results.txt", "w") as file:
#     file.write(f"layer5 R² score: {r2}\n")
#     file.write(f"layer5 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer5_pred_data = layer5_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer5_pred_data.npy',layer5_pred_data)
# np.save(r'./layer5/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer5_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer5_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer6

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy')
layer2_fanyan = np.load(r'./layer2_pred_data.npy')
layer3_fanyan = np.load(r'./layer3_pred_data.npy')
layer4_fanyan = np.load(r'./layer4_pred_data.npy')
layer5_fanyan = np.load(r'./layer5_pred_data.npy')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan,layer3_fanyan,layer4_fanyan,layer5_fanyan), axis=-1)
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,5:6] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer6_st_no.keras',      safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:10]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer6_st = scaler_l.inverse_transform(result)
y_test_layer6_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer6_unscaled[:,0],predict_layer6_st[:,0])
print('layer6 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer6_unscaled[:,0]-predict_layer6_st[:,0])))
print('layer6 Test mae: %.3f' % mae)

layer6_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer6_pred_data[non_nan_mask] = predict_layer6_st
true_data[non_nan_mask] = y_test_layer6_unscaled

layer6_pred_data[layer6_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer6/results.txt", "w") as file:
#     file.write(f"layer6 R² score: {r2}\n")
#     file.write(f"layer6 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer6_pred_data = layer6_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer6_pred_data.npy',layer6_pred_data)
# np.save(r'./layer6/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer6_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer6_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer7

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy')
layer2_fanyan = np.load(r'./layer2_pred_data.npy')
layer3_fanyan = np.load(r'./layer3_pred_data.npy')
layer4_fanyan = np.load(r'./layer4_pred_data.npy')
layer5_fanyan = np.load(r'./layer5_pred_data.npy')
layer6_fanyan = np.load(r'./layer6_pred_data.npy')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan,layer3_fanyan,layer4_fanyan,layer5_fanyan,layer6_fanyan), axis=-1)
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,6:7] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer7_st_no.keras',      safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:11]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer7_st = scaler_l.inverse_transform(result)
y_test_layer7_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer7_unscaled[:,0],predict_layer7_st[:,0])
print('layer7 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer7_unscaled[:,0]-predict_layer7_st[:,0])))
print('layer7 Test mae: %.3f' % mae)

layer7_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer7_pred_data[non_nan_mask] = predict_layer7_st
true_data[non_nan_mask] = y_test_layer7_unscaled

layer7_pred_data[layer7_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer7/results.txt", "w") as file:
#     file.write(f"layer7 R² score: {r2}\n")
#     file.write(f"layer7 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer7_pred_data = layer7_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer7_pred_data.npy',layer7_pred_data)
# np.save(r'./layer7/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer7_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer7_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer8

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy')
layer2_fanyan = np.load(r'./layer2_pred_data.npy')
layer3_fanyan = np.load(r'./layer3_pred_data.npy')
layer4_fanyan = np.load(r'./layer4_pred_data.npy')
layer5_fanyan = np.load(r'./layer5_pred_data.npy')
layer6_fanyan = np.load(r'./layer6_pred_data.npy')
layer7_fanyan = np.load(r'./layer7_pred_data.npy')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan,layer3_fanyan,layer4_fanyan,layer5_fanyan,layer6_fanyan,layer7_fanyan), axis=-1)
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,7:8] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer8_st_no.keras',      safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:12]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer8_st = scaler_l.inverse_transform(result)
y_test_layer8_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer8_unscaled[:,0],predict_layer8_st[:,0])
print('layer8 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer8_unscaled[:,0]-predict_layer8_st[:,0])))
print('layer8 Test mae: %.3f' % mae)

layer8_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer8_pred_data[non_nan_mask] = predict_layer8_st
true_data[non_nan_mask] = y_test_layer8_unscaled

layer8_pred_data[layer8_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer8/results.txt", "w") as file:
#     file.write(f"layer8 R² score: {r2}\n")
#     file.write(f"layer8 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer8_pred_data = layer8_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer8_pred_data.npy',layer8_pred_data)
# np.save(r'./layer8/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer8_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer8_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer9

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy', mmap_mode='r')
layer2_fanyan = np.load(r'./layer2_pred_data.npy', mmap_mode='r')
layer3_fanyan = np.load(r'./layer3_pred_data.npy', mmap_mode='r')
layer4_fanyan = np.load(r'./layer4_pred_data.npy', mmap_mode='r')
layer5_fanyan = np.load(r'./layer5_pred_data.npy', mmap_mode='r')
layer6_fanyan = np.load(r'./layer6_pred_data.npy', mmap_mode='r')
layer7_fanyan = np.load(r'./layer7_pred_data.npy', mmap_mode='r')
layer8_fanyan = np.load(r'./layer8_pred_data.npy', mmap_mode='r')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan,layer3_fanyan,layer4_fanyan,layer5_fanyan,layer6_fanyan,layer7_fanyan,layer8_fanyan), axis=-1)
data_underwater = data_underwater[0:132,:,:,:]
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,8:9] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer9_st_no.keras', safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:13]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer9_st = scaler_l.inverse_transform(result)
y_test_layer9_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer9_unscaled[:,0],predict_layer9_st[:,0])
print('layer9 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer9_unscaled[:,0]-predict_layer9_st[:,0])))
print('layer9 Test mae: %.3f' % mae)

layer9_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer9_pred_data[non_nan_mask] = predict_layer9_st
true_data[non_nan_mask] = y_test_layer9_unscaled

layer9_pred_data[layer9_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer9/results.txt", "w") as file:
#     file.write(f"layer9 R² score: {r2}\n")
#     file.write(f"layer9 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer9_pred_data = layer9_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer9_pred_data.npy',layer9_pred_data)
# np.save(r'./layer9/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer9_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer9_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer 10

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy', mmap_mode='r')
layer2_fanyan = np.load(r'./layer2_pred_data.npy', mmap_mode='r')
layer3_fanyan = np.load(r'./layer3_pred_data.npy', mmap_mode='r')
layer4_fanyan = np.load(r'./layer4_pred_data.npy', mmap_mode='r')
layer5_fanyan = np.load(r'./layer5_pred_data.npy', mmap_mode='r')
layer6_fanyan = np.load(r'./layer6_pred_data.npy', mmap_mode='r')
layer7_fanyan = np.load(r'./layer7_pred_data.npy', mmap_mode='r')
layer8_fanyan = np.load(r'./layer8_pred_data.npy', mmap_mode='r')
layer9_fanyan = np.load(r'./layer9_pred_data.npy', mmap_mode='r')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan,layer3_fanyan,layer4_fanyan,layer5_fanyan,layer6_fanyan,layer7_fanyan,layer8_fanyan,layer9_fanyan), axis=-1)
data_underwater = data_underwater[0:132,:,:,:]
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,9:10] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer10_st_no.keras', safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:14]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer10_st = scaler_l.inverse_transform(result)
y_test_layer10_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer10_unscaled[:,0],predict_layer10_st[:,0])
print('layer10 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer10_unscaled[:,0]-predict_layer10_st[:,0])))
print('layer10 Test mae: %.3f' % mae)

layer10_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer10_pred_data[non_nan_mask] = predict_layer10_st
true_data[non_nan_mask] = y_test_layer10_unscaled

layer10_pred_data[layer10_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer10/results.txt", "w") as file:
#     file.write(f"layer10 R² score: {r2}\n")
#     file.write(f"layer10 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer10_pred_data = layer10_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer10_pred_data.npy',layer10_pred_data)
# np.save(r'./layer10/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer10_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer10_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer 11

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy', mmap_mode='r')
layer2_fanyan = np.load(r'./layer2_pred_data.npy', mmap_mode='r')
layer3_fanyan = np.load(r'./layer3_pred_data.npy', mmap_mode='r')
layer4_fanyan = np.load(r'./layer4_pred_data.npy', mmap_mode='r')
layer5_fanyan = np.load(r'./layer5_pred_data.npy', mmap_mode='r')
layer6_fanyan = np.load(r'./layer6_pred_data.npy', mmap_mode='r')
layer7_fanyan = np.load(r'./layer7_pred_data.npy', mmap_mode='r')
layer8_fanyan = np.load(r'./layer8_pred_data.npy', mmap_mode='r')
layer9_fanyan = np.load(r'./layer9_pred_data.npy', mmap_mode='r')
layer10_fanyan = np.load(r'./layer10_pred_data.npy', mmap_mode='r')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan,layer3_fanyan,layer4_fanyan,layer5_fanyan,layer6_fanyan,layer7_fanyan,layer8_fanyan,layer9_fanyan,
                                  layer10_fanyan), axis=-1)
data_underwater = data_underwater[0:132,:,:,:]
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,10:11] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer11_st_no.keras', safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:15]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer11_st = scaler_l.inverse_transform(result)
y_test_layer11_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer11_unscaled[:,0],predict_layer11_st[:,0])
print('layer11 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer11_unscaled[:,0]-predict_layer11_st[:,0])))
print('layer11 Test mae: %.3f' % mae)

layer11_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer11_pred_data[non_nan_mask] = predict_layer11_st
true_data[non_nan_mask] = y_test_layer11_unscaled

layer11_pred_data[layer11_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer11/results.txt", "w") as file:
#     file.write(f"layer11 R² score: {r2}\n")
#     file.write(f"layer11 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer11_pred_data = layer11_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer11_pred_data.npy',layer11_pred_data)
# np.save(r'./layer11/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer11_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer11_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer 12

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy', mmap_mode='r')
layer2_fanyan = np.load(r'./layer2_pred_data.npy', mmap_mode='r')
layer3_fanyan = np.load(r'./layer3_pred_data.npy', mmap_mode='r')
layer4_fanyan = np.load(r'./layer4_pred_data.npy', mmap_mode='r')
layer5_fanyan = np.load(r'./layer5_pred_data.npy', mmap_mode='r')
layer6_fanyan = np.load(r'./layer6_pred_data.npy', mmap_mode='r')
layer7_fanyan = np.load(r'./layer7_pred_data.npy', mmap_mode='r')
layer8_fanyan = np.load(r'./layer8_pred_data.npy', mmap_mode='r')
layer9_fanyan = np.load(r'./layer9_pred_data.npy', mmap_mode='r')
layer10_fanyan = np.load(r'./layer10_pred_data.npy', mmap_mode='r')
layer11_fanyan = np.load(r'./layer11_pred_data.npy', mmap_mode='r')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan,layer3_fanyan,layer4_fanyan,layer5_fanyan,layer6_fanyan,layer7_fanyan,layer8_fanyan,layer9_fanyan,
                                  layer10_fanyan,layer11_fanyan), axis=-1)
data_underwater = data_underwater[0:132,:,:,:]
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,11:12] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer12_st_no.keras', safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:16]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer12_st = scaler_l.inverse_transform(result)
y_test_layer12_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer12_unscaled[:,0],predict_layer12_st[:,0])
print('layer12 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer12_unscaled[:,0]-predict_layer12_st[:,0])))
print('layer12 Test mae: %.3f' % mae)

layer12_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer12_pred_data[non_nan_mask] = predict_layer12_st
true_data[non_nan_mask] = y_test_layer12_unscaled

layer12_pred_data[layer12_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer12/results.txt", "w") as file:
#     file.write(f"layer12 R² score: {r2}\n")
#     file.write(f"layer12 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer12_pred_data = layer12_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer12_pred_data.npy',layer12_pred_data)
# np.save(r'./layer12/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer12_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer12_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer13

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy', mmap_mode='r')
layer2_fanyan = np.load(r'./layer2_pred_data.npy', mmap_mode='r')
layer3_fanyan = np.load(r'./layer3_pred_data.npy', mmap_mode='r')
layer4_fanyan = np.load(r'./layer4_pred_data.npy', mmap_mode='r')
layer5_fanyan = np.load(r'./layer5_pred_data.npy', mmap_mode='r')
layer6_fanyan = np.load(r'./layer6_pred_data.npy', mmap_mode='r')
layer7_fanyan = np.load(r'./layer7_pred_data.npy', mmap_mode='r')
layer8_fanyan = np.load(r'./layer8_pred_data.npy', mmap_mode='r')
layer9_fanyan = np.load(r'./layer9_pred_data.npy', mmap_mode='r')
layer10_fanyan = np.load(r'./layer10_pred_data.npy', mmap_mode='r')
layer11_fanyan = np.load(r'./layer11_pred_data.npy', mmap_mode='r')
layer12_fanyan = np.load(r'./layer12_pred_data.npy', mmap_mode='r')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan,layer3_fanyan,layer4_fanyan,layer5_fanyan,layer6_fanyan,layer7_fanyan,layer8_fanyan,layer9_fanyan,
                                  layer10_fanyan,layer11_fanyan,layer12_fanyan), axis=-1)
data_underwater = data_underwater[0:132,:,:,:]
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,12:13] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer13_st_no.keras', safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:17]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer13_st = scaler_l.inverse_transform(result)
y_test_layer13_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer13_unscaled[:,0],predict_layer13_st[:,0])
print('layer13 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer13_unscaled[:,0]-predict_layer13_st[:,0])))
print('layer13 Test mae: %.3f' % mae)

layer13_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer13_pred_data[non_nan_mask] = predict_layer13_st
true_data[non_nan_mask] = y_test_layer13_unscaled

layer13_pred_data[layer13_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer13/results.txt", "w") as file:
#     file.write(f"layer13 R² score: {r2}\n")
#     file.write(f"layer13 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer13_pred_data = layer13_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer13_pred_data.npy',layer13_pred_data)
# np.save(r'./layer13/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer13_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer13_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer14

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy', mmap_mode='r')
layer2_fanyan = np.load(r'./layer2_pred_data.npy', mmap_mode='r')
layer3_fanyan = np.load(r'./layer3_pred_data.npy', mmap_mode='r')
layer4_fanyan = np.load(r'./layer4_pred_data.npy', mmap_mode='r')
layer5_fanyan = np.load(r'./layer5_pred_data.npy', mmap_mode='r')
layer6_fanyan = np.load(r'./layer6_pred_data.npy', mmap_mode='r')
layer7_fanyan = np.load(r'./layer7_pred_data.npy', mmap_mode='r')
layer8_fanyan = np.load(r'./layer8_pred_data.npy', mmap_mode='r')
layer9_fanyan = np.load(r'./layer9_pred_data.npy', mmap_mode='r')
layer10_fanyan = np.load(r'./layer10_pred_data.npy', mmap_mode='r')
layer11_fanyan = np.load(r'./layer11_pred_data.npy', mmap_mode='r')
layer12_fanyan = np.load(r'./layer12_pred_data.npy', mmap_mode='r')
layer13_fanyan = np.load(r'./layer13_pred_data.npy', mmap_mode='r')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan,layer3_fanyan,layer4_fanyan,layer5_fanyan,layer6_fanyan,layer7_fanyan,layer8_fanyan,layer9_fanyan,
                                  layer10_fanyan,layer11_fanyan,layer12_fanyan,layer13_fanyan), axis=-1)
data_underwater = data_underwater[0:132,:,:,:]
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,13:14] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer14_st_no.keras', safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:18]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer14_st = scaler_l.inverse_transform(result)
y_test_layer14_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer14_unscaled[:,0],predict_layer14_st[:,0])
print('layer14 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer14_unscaled[:,0]-predict_layer14_st[:,0])))
print('layer14 Test mae: %.3f' % mae)

layer14_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer14_pred_data[non_nan_mask] = predict_layer14_st
true_data[non_nan_mask] = y_test_layer14_unscaled

layer14_pred_data[layer14_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer14/results.txt", "w") as file:
#     file.write(f"layer14 R² score: {r2}\n")
#     file.write(f"layer14 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer14_pred_data = layer14_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer14_pred_data.npy',layer14_pred_data)
# np.save(r'./layer14/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer14_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer14_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer15

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy', mmap_mode='r')
layer2_fanyan = np.load(r'./layer2_pred_data.npy', mmap_mode='r')
layer3_fanyan = np.load(r'./layer3_pred_data.npy', mmap_mode='r')
layer4_fanyan = np.load(r'./layer4_pred_data.npy', mmap_mode='r')
layer5_fanyan = np.load(r'./layer5_pred_data.npy', mmap_mode='r')
layer6_fanyan = np.load(r'./layer6_pred_data.npy', mmap_mode='r')
layer7_fanyan = np.load(r'./layer7_pred_data.npy', mmap_mode='r')
layer8_fanyan = np.load(r'./layer8_pred_data.npy', mmap_mode='r')
layer9_fanyan = np.load(r'./layer9_pred_data.npy', mmap_mode='r')
layer10_fanyan = np.load(r'./layer10_pred_data.npy', mmap_mode='r')
layer11_fanyan = np.load(r'./layer11_pred_data.npy', mmap_mode='r')
layer12_fanyan = np.load(r'./layer12_pred_data.npy', mmap_mode='r')
layer13_fanyan = np.load(r'./layer13_pred_data.npy', mmap_mode='r')
layer14_fanyan = np.load(r'./layer14_pred_data.npy', mmap_mode='r')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan,layer3_fanyan,layer4_fanyan,layer5_fanyan,layer6_fanyan,layer7_fanyan,layer8_fanyan,layer9_fanyan,
                                  layer10_fanyan,layer11_fanyan,layer12_fanyan,layer13_fanyan,layer14_fanyan), axis=-1)
data_underwater = data_underwater[0:132,:,:,:]
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,14:15] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer15_st_no.keras', safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:19]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer15_st = scaler_l.inverse_transform(result)
y_test_layer15_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer15_unscaled[:,0],predict_layer15_st[:,0])
print('layer15 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer15_unscaled[:,0]-predict_layer15_st[:,0])))
print('layer15 Test mae: %.3f' % mae)

layer15_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer15_pred_data[non_nan_mask] = predict_layer15_st
true_data[non_nan_mask] = y_test_layer15_unscaled

layer15_pred_data[layer15_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer15/results.txt", "w") as file:
#     file.write(f"layer15 R² score: {r2}\n")
#     file.write(f"layer15 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer15_pred_data = layer15_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer15_pred_data.npy',layer15_pred_data)
# np.save(r'./layer15/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer15_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer15_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer16

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy', mmap_mode='r')
layer2_fanyan = np.load(r'./layer2_pred_data.npy', mmap_mode='r')
layer3_fanyan = np.load(r'./layer3_pred_data.npy', mmap_mode='r')
layer4_fanyan = np.load(r'./layer4_pred_data.npy', mmap_mode='r')
layer5_fanyan = np.load(r'./layer5_pred_data.npy', mmap_mode='r')
layer6_fanyan = np.load(r'./layer6_pred_data.npy', mmap_mode='r')
layer7_fanyan = np.load(r'./layer7_pred_data.npy', mmap_mode='r')
layer8_fanyan = np.load(r'./layer8_pred_data.npy', mmap_mode='r')
layer9_fanyan = np.load(r'./layer9_pred_data.npy', mmap_mode='r')
layer10_fanyan = np.load(r'./layer10_pred_data.npy', mmap_mode='r')
layer11_fanyan = np.load(r'./layer11_pred_data.npy', mmap_mode='r')
layer12_fanyan = np.load(r'./layer12_pred_data.npy', mmap_mode='r')
layer13_fanyan = np.load(r'./layer13_pred_data.npy', mmap_mode='r')
layer14_fanyan = np.load(r'./layer14_pred_data.npy', mmap_mode='r')
layer15_fanyan = np.load(r'./layer15_pred_data.npy', mmap_mode='r')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan,layer3_fanyan,layer4_fanyan,layer5_fanyan,layer6_fanyan,layer7_fanyan,layer8_fanyan,layer9_fanyan,
                                  layer10_fanyan,layer11_fanyan,layer12_fanyan,layer13_fanyan,layer14_fanyan,layer15_fanyan), axis=-1)
data_underwater = data_underwater[0:132,:,:,:]
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,15:16] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer16_st_no.keras', safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:20]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer16_st = scaler_l.inverse_transform(result)
y_test_layer16_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer16_unscaled[:,0],predict_layer16_st[:,0])
print('layer16 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer16_unscaled[:,0]-predict_layer16_st[:,0])))
print('layer16 Test mae: %.3f' % mae)

layer16_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer16_pred_data[non_nan_mask] = predict_layer16_st
true_data[non_nan_mask] = y_test_layer16_unscaled

layer16_pred_data[layer16_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer16/results.txt", "w") as file:
#     file.write(f"layer16 R² score: {r2}\n")
#     file.write(f"layer16 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer16_pred_data = layer16_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer16_pred_data.npy',layer16_pred_data)
# np.save(r'./layer16/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer16_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer16_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer17

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy', mmap_mode='r')
layer2_fanyan = np.load(r'./layer2_pred_data.npy', mmap_mode='r')
layer3_fanyan = np.load(r'./layer3_pred_data.npy', mmap_mode='r')
layer4_fanyan = np.load(r'./layer4_pred_data.npy', mmap_mode='r')
layer5_fanyan = np.load(r'./layer5_pred_data.npy', mmap_mode='r')
layer6_fanyan = np.load(r'./layer6_pred_data.npy', mmap_mode='r')
layer7_fanyan = np.load(r'./layer7_pred_data.npy', mmap_mode='r')
layer8_fanyan = np.load(r'./layer8_pred_data.npy', mmap_mode='r')
layer9_fanyan = np.load(r'./layer9_pred_data.npy', mmap_mode='r')
layer10_fanyan = np.load(r'./layer10_pred_data.npy', mmap_mode='r')
layer11_fanyan = np.load(r'./layer11_pred_data.npy', mmap_mode='r')
layer12_fanyan = np.load(r'./layer12_pred_data.npy', mmap_mode='r')
layer13_fanyan = np.load(r'./layer13_pred_data.npy', mmap_mode='r')
layer14_fanyan = np.load(r'./layer14_pred_data.npy', mmap_mode='r')
layer15_fanyan = np.load(r'./layer15_pred_data.npy', mmap_mode='r')
layer16_fanyan = np.load(r'./layer16_pred_data.npy', mmap_mode='r')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan,layer3_fanyan,layer4_fanyan,layer5_fanyan,layer6_fanyan,layer7_fanyan,layer8_fanyan,layer9_fanyan,
                                  layer10_fanyan,layer11_fanyan,layer12_fanyan,layer13_fanyan,layer14_fanyan,layer15_fanyan,layer16_fanyan
                                  ), axis=-1)
data_underwater = data_underwater[0:132,:,:,:]
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,16:17] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer17_st_no.keras', safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:21]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer17_st = scaler_l.inverse_transform(result)
y_test_layer17_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer17_unscaled[:,0],predict_layer17_st[:,0])
print('layer17 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer17_unscaled[:,0]-predict_layer17_st[:,0])))
print('layer17 Test mae: %.3f' % mae)

layer17_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer17_pred_data[non_nan_mask] = predict_layer17_st
true_data[non_nan_mask] = y_test_layer17_unscaled

layer17_pred_data[layer17_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer17/results.txt", "w") as file:
#     file.write(f"layer17 R² score: {r2}\n")
#     file.write(f"layer17 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer17_pred_data = layer17_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer17_pred_data.npy',layer17_pred_data)
# np.save(r'./layer17/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer17_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer17_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer18

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy', mmap_mode='r')
layer2_fanyan = np.load(r'./layer2_pred_data.npy', mmap_mode='r')
layer3_fanyan = np.load(r'./layer3_pred_data.npy', mmap_mode='r')
layer4_fanyan = np.load(r'./layer4_pred_data.npy', mmap_mode='r')
layer5_fanyan = np.load(r'./layer5_pred_data.npy', mmap_mode='r')
layer6_fanyan = np.load(r'./layer6_pred_data.npy', mmap_mode='r')
layer7_fanyan = np.load(r'./layer7_pred_data.npy', mmap_mode='r')
layer8_fanyan = np.load(r'./layer8_pred_data.npy', mmap_mode='r')
layer9_fanyan = np.load(r'./layer9_pred_data.npy', mmap_mode='r')
layer10_fanyan = np.load(r'./layer10_pred_data.npy', mmap_mode='r')
layer11_fanyan = np.load(r'./layer11_pred_data.npy', mmap_mode='r')
layer12_fanyan = np.load(r'./layer12_pred_data.npy', mmap_mode='r')
layer13_fanyan = np.load(r'./layer13_pred_data.npy', mmap_mode='r')
layer14_fanyan = np.load(r'./layer14_pred_data.npy', mmap_mode='r')
layer15_fanyan = np.load(r'./layer15_pred_data.npy', mmap_mode='r')
layer16_fanyan = np.load(r'./layer16_pred_data.npy', mmap_mode='r')
layer17_fanyan = np.load(r'./layer17_pred_data.npy', mmap_mode='r')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan,layer3_fanyan,layer4_fanyan,layer5_fanyan,layer6_fanyan,layer7_fanyan,layer8_fanyan,layer9_fanyan,
                                  layer10_fanyan,layer11_fanyan,layer12_fanyan,layer13_fanyan,layer14_fanyan,layer15_fanyan,layer16_fanyan,
                                  layer17_fanyan), axis=-1)
data_underwater = data_underwater[0:132,:,:,:]
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,17:18] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer18_st_no.keras', safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:22]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer18_st = scaler_l.inverse_transform(result)
y_test_layer18_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer18_unscaled[:,0],predict_layer18_st[:,0])
print('layer18 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer18_unscaled[:,0]-predict_layer18_st[:,0])))
print('layer18 Test mae: %.3f' % mae)

layer18_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer18_pred_data[non_nan_mask] = predict_layer18_st
true_data[non_nan_mask] = y_test_layer18_unscaled

layer18_pred_data[layer18_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer18/results.txt", "w") as file:
#     file.write(f"layer18 R² score: {r2}\n")
#     file.write(f"layer18 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer18_pred_data = layer18_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer18_pred_data.npy',layer18_pred_data)
# np.save(r'./layer18/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer18_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer18_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# Layer19

In [ ]:
data_surface = data_surface_processed[0:132,:,:,:]

data_input = data_surface

data_st = data_st_processed[0:132,:,:,:]
layer1_fanyan = np.load(r'./layer1_pred_data.npy', mmap_mode='r')
layer2_fanyan = np.load(r'./layer2_pred_data.npy', mmap_mode='r')
layer3_fanyan = np.load(r'./layer3_pred_data.npy', mmap_mode='r')
layer4_fanyan = np.load(r'./layer4_pred_data.npy', mmap_mode='r')
layer5_fanyan = np.load(r'./layer5_pred_data.npy', mmap_mode='r')
layer6_fanyan = np.load(r'./layer6_pred_data.npy', mmap_mode='r')
layer7_fanyan = np.load(r'./layer7_pred_data.npy', mmap_mode='r')
layer8_fanyan = np.load(r'./layer8_pred_data.npy', mmap_mode='r')
layer9_fanyan = np.load(r'./layer9_pred_data.npy', mmap_mode='r')
layer10_fanyan = np.load(r'./layer10_pred_data.npy', mmap_mode='r')
layer11_fanyan = np.load(r'./layer11_pred_data.npy', mmap_mode='r')
layer12_fanyan = np.load(r'./layer12_pred_data.npy', mmap_mode='r')
layer13_fanyan = np.load(r'./layer13_pred_data.npy', mmap_mode='r')
layer14_fanyan = np.load(r'./layer14_pred_data.npy', mmap_mode='r')
layer15_fanyan = np.load(r'./layer15_pred_data.npy', mmap_mode='r')
layer16_fanyan = np.load(r'./layer16_pred_data.npy', mmap_mode='r')
layer17_fanyan = np.load(r'./layer17_pred_data.npy', mmap_mode='r')
layer18_fanyan = np.load(r'./layer18_pred_data.npy', mmap_mode='r')
data_underwater = np.concatenate((layer1_fanyan,layer2_fanyan,layer3_fanyan,layer4_fanyan,layer5_fanyan,layer6_fanyan,layer7_fanyan,layer8_fanyan,layer9_fanyan,
                                  layer10_fanyan,layer11_fanyan,layer12_fanyan,layer13_fanyan,layer14_fanyan,layer15_fanyan,layer16_fanyan,
                                  layer17_fanyan,layer18_fanyan), axis=-1)
data_underwater = data_underwater[0:132,:,:,:]
print(data_surface.shape,data_st.shape)

out_data_y = data_st_processed[0:132,:,:,18:19] #(132,  100，160， 1)
print(out_data_y.shape)

In [ ]:
time_size, lat_size, lon_size, channel1 = data_input.shape   # （132，120，120，5）
#print( time_size, lat_size, lon_size,channel)
# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size + 1) // step_size  
out_lon_size = (lon_size - box_size + 1) // step_size  

#输出数组 

channel2 =  data_underwater.shape[3]
pro_data = np.zeros((time_size, lat_size , lon_size, channel2)) #(132, 208, 208, 6)

for t in range(time_size):
    # 2

    pro1 = np.concatenate((np.zeros((4 , out_lon_size, channel2)),data_underwater[t,:,:,:],np.zeros((4 , out_lon_size, channel2)))) #9*9
    # pro1 = np.concatenate((np.zeros((5 , out_lon_size, channel)),data_surface[t,:,:,:],np.zeros((5 , out_lon_size, channel)))) #

    pro1 = np.transpose(pro1,(1,0,2))
    # print(pro1.shape)

    pro2 = np.concatenate((np.zeros((4 , out_lat_size+box_size-1, channel2)),pro1[:,:,:],np.zeros((4 , out_lat_size+box_size-1, channel2)))) #9*9
    # pro2 = np.concatenate((np.zeros((5 , out_lon_size+box_size-1, channel)),pro1[:,:,:],np.zeros((5 , out_lon_size+box_size-1, channel)))) #

    pro2 = np.transpose(pro2,(1,0,2))
    # print(pro2.shape) 
    pro_data[t,:,:,:] = pro2

pro_data_out = np.concatenate((data_input, pro_data), axis=-1)
channel = channel1 + channel2
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 100，160, 9,9,5)

print(out_data.shape)

# 遍历时间步、纬度和经度
for t in range(time_size):
    for lat in range(0, lat_size - box_size + 1 , step_size):
        for lon in range(0, lon_size - box_size + 1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            # 获取当前矩形框的地图数据
            box_data = pro_data_out[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []   

# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,channel))#（132，16000，9，9，5）
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,4:5]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（16000，81）    # 其中true 代表Nan
print(nan_mask_out_data_one.shape)
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,1)

nan_mask_out_data_y = np.isnan(out_data_y_reshape)
nan_mask_out_data_y_one = nan_mask_out_data_y[:,0,0]   
nan_mask2 = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <1):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask2.append(True)
    else:
        nan_mask2.append(False)

nan_mask = np.array(nan_mask)        # 转换为 NumPy 数组
nan_mask2 = np.array(nan_mask2)      # 转换为 NumPy 数组

# 使用 & 操作（注意：这里是逐元素与，不是逻辑与）
combined_mask = nan_mask & nan_mask2  # 正确：NumPy 数组支持 &

# 应用筛选
out_data_reshape = out_data_reshape[combined_mask]
out_data_y_reshape = out_data_y_reshape[combined_mask]

out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)


x_test = out_data_reshape
y_test = out_data_y_reshape

# 将前两个维度进行合并  时间*(lat*lon)
x_test = np.reshape(x_test,(-1,9,9,channel))
y_test = np.reshape(y_test,(-1,1))


x_test_shape = x_test.shape
y_test_shape = y_test.shape

scaler_f = StandardScaler()
x_test = scaler_f.fit_transform(x_test.reshape(-1,x_test_shape[3]))

scaler_l = StandardScaler()
y_test = scaler_l.fit_transform(y_test.reshape(-1,y_test_shape[1])) 

x_test = np.reshape(x_test,(-1,9,9,channel))#111792，9，9，5
y_test = np.reshape(y_test,(-1,1))#111792，15
print(scaler_l.mean_.shape,scaler_l.scale_.shape)

In [ ]:
# 模型预测
result = [] 

best_model = tf.keras.models.load_model('./CNN_model_layer19_st_no.keras', safe_mode=False)
testPred = best_model.predict([x_test[:,:,:,0:5], x_test[:,:,:,5:23]])
result.append(testPred)

result = np.array(result)
result = result.reshape(-1,1)

y_test = y_test.reshape(-1,1)
predict_layer19_st = scaler_l.inverse_transform(result)
y_test_layer19_unscaled = scaler_l.inverse_transform(y_test)

r2 = r2_score(y_test_layer19_unscaled[:,0],predict_layer19_st[:,0])
print('layer19 R² score:',r2)

mae =  np.mean(np.sqrt(np.square(y_test_layer19_unscaled[:,0]-predict_layer19_st[:,0])))
print('layer19 Test mae: %.3f' % mae)

layer19_pred_data = np.zeros((132, 16000, 1 ))  # out_data_reshape 的当前形状
true_data = np.zeros((132, 16000, 1 )) 
non_nan_mask = np.broadcast_to(combined_mask, (132, 16000))
layer19_pred_data[non_nan_mask] = predict_layer19_st
true_data[non_nan_mask] = y_test_layer19_unscaled

layer19_pred_data[layer19_pred_data == 0] = np.nan
true_data[true_data == 0] = np.nan
# 打开一个文件并写入内容
# with open("./layer19/results.txt", "w") as file:
#     file.write(f"layer19 R² score: {r2}\n")
#     file.write(f"layer19 Test mse: {mae:.3f}\n")

# print("Results written to results.txt")

layer19_pred_data = layer19_pred_data.reshape(132, 100,-1,1)
true_data = true_data.reshape(132, 100,-1,1)

np.save(r'./layer19_pred_data.npy',layer19_pred_data)
# np.save(r'./layer19/true_data.npy',true_data)

plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(layer19_pred_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('Pred')
print(layer19_pred_data[0,:,:,0])
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/testPred_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执
plt.show()
plt.figure(figsize=(3.6, 3.6), dpi = 100)
plt.imshow(true_data[0,:,:,0])              #执行这一行后并不会立即看到图像，这一行更像是将depthmap载入到plt里
plt.title('True')
# plt.savefig('D:/Postgra/South_Ocean/Conv3D+LSTM/model5_18/y_test_with_nan5_18.jpg')       #执行后可以将文件保存为jpg格式图像，可以双击直接查看。也可以不执行
plt.show()
print(true_data[0,:,:,0])

gc.collect()

# End